# Lab 3: Functional programming, and declarative patterns


__Student:__ olojo524

__Student:__ praxx536

Disclaimer: Functional programming in Python does not always lead to the fastest possible code, and is often not considered the *pythonic* approach. However, functional programming is the basis for many concurrent systems (the MapReduce programming model which many big data systems, e.g. Hadoop, relies on gets its name from the *map* and *reduce* functions mentioned below). Python is a multi-paradigm language, and functional programming is one of the main paradigms one can use. To understand how and when to do this, it is necessary to do things in a non-*pythonic* way in order to cover the basics.

## General instructions

In this lab there are some general rules you should keep in mind to make sure you are on the correct path in your solutions.

#### Rules
1. You are not allowed to use `while` or `for` statements unless this is explicitly allowed in the task.
2. You are not allowed to use global variables (other than for functions defined in the global environment).
3. Code stubs should be viewed as fixed, you are only allowed to add code, the only code you are allowed to change is `pass` statements, which you should remove.
4. You should refrain from using the `list` datatype unless otherwise specified and instead use `tuple`. One of the strengths of functional programming is its focus on immutable data types (this is why functional programming and concurrency goes so well together). Incidentally, one might find speedups when using the immutable tuples instead of lists.

#### Advice
1. Avoid local variables unless you are certain they are necessary, in most cases you won't need to use local variables. (altermatively, use local variables to your hearts content, but when your solution works, try to eliminate them, you should be able to eliminate most of them, over time, you might find that you don't need them.)

# 2 Recursion

As an introduction to linear recursion, read the introductory note on the course webpage. This might help explain terms that you may not know (even if the concept is previously known).

## 2.1 Linear recursion

a) Write a recursive function `sum_even(n)` that takes a natural number $n\geq 0$ and returns the sum of all even numbers $0,...,n$. It should be linear-recursive with delayed computations.

In [1]:
def sum_even(n):
    if n <= 0:
        return 0
    if not n % 2 == 0:
        return sum_even(n-1)
    else:
        return  sum_even(n-2) + n # 4, 2, 0

sum_even(4)


6

b) Write `sum_even_it(n)` according to the same specification. In this case, the solution should be tail recursive.

In [4]:
def sum_even_it(n, tail):
    
    if n <= 0:
        return(tail)
    
    if not n % 2 == 0:
        n, tail = n-1, tail # Update -1 if uneven

    else:
        n, tail = n-2, tail + n # Update -2 if even
    
    return sum_even_it(n, tail)

print(sum_even_it(4, 0))


6


c) We can of course express this in a declarative and Pythonic way, which is non-recursive. Write a function `sum_even_py` which returns the same result as above, but using a comprehension or filter/map/reduce construct.

In [3]:
def sum_even_py(n):
    return(sum(i for i in range(n+1) if i %2 == 0))

print(sum_even_py(4))


6


## 3 Higher order functions (HOF)
 
A _higher-order function_ is a function which operates on other functions. What this means exactly is disputed, but we will call any function which returns a function or takes a function as an argument a higher-order function. (Conversely, a function neither taking another function as input nor returning a function we will refer to as a _first-order function_)

In R you have encountered these when, for instance, using the `apply` family of functions, which are all versions of what is called a `map` function in functional programming (see below).

When using higher-order functions, it is often useful to create simple anonymous functions at the place in the code where they are used, rather than defining a new named function in one place only to call it in a single other place. In R, all functions are created in this way with the `function` keyword, but they are usually assigned to global names with standard assignment (`<-`). Python provides similar functionality using the `lambda` keyword (name inspired by Alonzo Church's [$\lambda$-calculus](https://www.youtube.com/watch?v=eis11j_iGMs) which has inspired much of functional programming) with which we can create anonymous functions. Of course, we can also pass named functions to higher-order functions, which is usually the case when the function is predefined, general enough to be used in more than one place, or complex enough to warrant separate definition and documentation for the sake of clarity.

## 3.1 The three standard functions `map`, `reduce` and `filter`

There are three standard cases which are widely applicable and many other higher-order functions are special cases or combinations of these. They are: `map`, apply a function on each element in a sequence, `filter`, keep (or conversely, remove) elements from a sequence according to some condition, and `reduce`, combine the elements in a sequence. The `map` function takes a sequence and a function (usually of 1 parameter) which is to be applied to each element of the sequence and might return anything, this function is assumed not to have side effects. The `filter` function takes a function (usually of 1 parameter) which returns a boolean value used to indicate which elements are to be kept. The `reduce` function takes a function (usually of 2 parameters) which is used to combine the elements in the sequence.

In Python, `map` and `filter` are standard built-in functions. Since Python 3, the `reduce` function needs to be imported from the `functools` module.

Many more advanced functions, of any order, can be created by combining these three higher-order functions.

A note from last year: usually, the `reduce` function is more difficult to grasp than `map` and `filter` but I found this blog-post by André Burgaud to be a nice introduction to `reduce`. Note that Burgaud talks about the more general _fold_ concept rather than `reduce`, which is a special case of fold often called _left fold_ (this is covered in more detail in the post). https://www.burgaud.com/foldl-foldr-python/

a) Implement a function `mysum` which computes the sum of a list or tuple of numbers using the reduce function and a lambda function.

In [6]:
from functools import reduce
def mysum(seq):

    return(reduce(lambda x, y: x+y, seq))

mysum((4, 7, 1))

12

b) Implement a function `mylength` which uses `map` and `reduce` (or only `reduce`, if you feel like it) to compute the length of a sequence. The use of the `len` function is not allowed.

[Hint: You can use `map` to convert the input to something which can easily be `reduce`:d.]

In [7]:
def mylength(seq):
    return reduce(lambda x, y: x+y, map(lambda x: 1, seq))
    
print(mylength((4, 2, 5, 2, 5)))
print(mylength("test"))

5
4


## 3.2 Building your own higher order functions (HOF)

a) Re-implement the three basic functional helper functions `map`, `filter` and `reduce` **as purely functional recursive functions**. You may not express this as eg comprehensions; the task is to practice figuring out this type of logic.

Note that the built-in versions of these functions work on multiple sequences of equal length if supplied, however, you can assume a single sequence as second parameter, i.e. you can also skip the third parameter to reduce.

In [6]:
def mymap(f, seq):
    if seq:
        return (f(seq[0]), ) + mymap(f, seq[1:])
    return ()

mymap(lambda x:x**2, tuple(range(10)))  # Should return the tuple (0, 1, 4, 9, ..., 81).


(0, 1, 4, 9, 16, 25, 36, 49, 64, 81)

In [8]:
def myfilter(f, seq):
    if seq:
        if f(seq[0]):
            return (seq[0],) + myfilter(f, seq[1:])
        else:
            return myfilter(f, seq[1:])
    return ()

myfilter(lambda x : x%2 == 0, tuple(range(10)))  # Should return the tuple of even numbers (0, 2, 4, 6, 8)

(0, 2, 4, 6, 8)

You might note the similarities with how you implemented `sum_even`.

In [10]:
def myreduce(f, seq):
    if len(seq) == 1:
        return seq[0]
    return f(seq[-1], myreduce(f, seq[:-1]))

print(myreduce(lambda x, y: x*y*y, tuple(range(1,5))))  # Should return the number 1*2*3*4 = 24
# print(myreduce(lambda x, y: x*y*y, tuple(range(1,5)))) # Verification

576


## 3.3 Returning functions

The previous section covered functions which take other functions as input, but what about the opposite, functions returning functions as output?

a) Function composition is a common in both maths and programming. Write a function `compose` which takes two functions, $f$ and $g$, and produces the _composite_ function $f \circ g$, where $(f \circ g)(x) \Leftrightarrow f(g(x))$. Example use is given below.

In [9]:
from statistics import stdev, mean

def compose(f, g):
    return lambda x: f(g(x))

def myscale(vals):
    return [x/stdev(vals) for x in vals]

def myshift(vals):
    return [x-mean(vals) for x in vals]

standardize = compose(myscale, myshift)

print(standardize(range(-3, 8)))

[-1.507556722888818, -1.2060453783110545, -0.9045340337332909, -0.6030226891555273, -0.30151134457776363, 0.0, 0.30151134457776363, 0.6030226891555273, 0.9045340337332909, 1.2060453783110545, 1.507556722888818]


**Voluntary task (not required)**
Create a function `composition(*funs)` which takes a non-empty sequence of functions of one argument and returns their sequential composition. That is $composition(f_0,f_1, \ldots, f_n) = f_0 \circ f_1 \circ\ldots \circ f_n$. (The question of if $f\circ g \circ h$ should be read $f\circ (g\circ h)$ or $(f \circ g) \circ h$ is perfectly valid, but they turn out to be the same. That is, $\circ$ is associative.)

In [12]:
def composition(*funs):
    def composition2(f, g):
        return lambda x: f(g(x))
    return reduce(composition2, funs)

######## for testing ################

def double(x):
    return x * 2

def plus(x):
    return x + 1

def subtract(x):
    return x - 1


subtract_plus_double = composition(double, plus, subtract)
print(subtract_plus_double(5))

double_double_subtract = composition(subtract, double, double)
print(double_double_subtract(10))



10
39


Hint: Don't remember what can be found in `*funs`? Print it! Don't know how the values should be combined? Write out some simple example on paper.

Note: This task demonstrates the generality of our constructs. Previously we worked with sequences of numbers and the like. Now we lift this to the level of working with functions as values, and instead of using combinators which work on numbers, we use function combinators in conjunction with our known patterns.

### Voluntary task: pipelining

When doing data analysis, one very important part is pre-processing. Often, data goes through a number of steps of preprocessing, sometimes called a pipeline. The function composition example above can be seen as a special case of such a pipeline for only two functions. By clever use of higher order functions, we can build a pipeline function which takes a list or tuple of data transforming functions and creates a function which applies these sequentially. Construct such a function called `make_pipeline`. In order to focus on the primary purpose of the `make_pipeline` function, we will perform a very simple set of transformations, increment each value by 1, take the absolute value, and then take the square root. Usage example and code for the `inc` function is supplied below.

You may want to use functions you have defined above.

In [11]:
from functools import reduce, partial
from math import sqrt 


""" Did not do"""

def make_pipeline(*funs):
    
    return lambda vals: ???

# We can even drop the lambda vals : bit, using partial
# evaluation (see the help for functools.partial!)

def inc(x):
    return x+1

def abs(x):
    return abs(x)

def sqrt(x):
    return x**2

make_pipeline(10, inc, abs, sqrt)
#pipeline = partial(make_pipeline, inc, abs, sqrt)
print(pipeline(5))

#tuple(pipeline(range(-5,5)))

NameError: name 'pipeline' is not defined

## 4. Simple declarative Pythonic patterns (involving higher order functions)

a) As preparation, create a named tuple type "coord" which has fields `x` and `y`.

In [14]:
# Add the requisite import statement here
from collections import namedtuple

coord = namedtuple('coord', ['x', 'y']) # your code here

five_three = coord(5,3)
assert five_three.x == 5, "first element is the x coordinate"
assert five_three.y == 3, "the second element is the y coordinate"

b) Generate a $10^7$ random coordinates, with $x$ and $y$ coordinates drawn uniformly from [-1000,1000]. Save the tuple of those with $x + y > 0$ as `rnd_coords`. How many are there (probably fewer than $10^7$)?

In [16]:
import random

rnd_coords = tuple(filter(lambda x: sum(x) > 0, 
((random.uniform(-1000, 1000), random.uniform(-1000, 1000)) for i in range(10**7))))

print(len(rnd_coords))
rnd_coords


5000800


((432.807610299976, 195.7467393917218),
 (617.982321782348, 794.7754858163771),
 (-548.38037309675, 830.2835162155325),
 (780.9607143564724, 682.4112475934332),
 (363.83071489786084, -22.800534994306986),
 (8.484873112383639, 175.4404692461692),
 (112.53308670177148, -78.60024315443195),
 (-764.6842508262612, 855.7484764030805),
 (969.4931186781907, -104.8539652763925),
 (159.86922191173358, 750.8069773752416),
 (-219.1333729032698, 642.2460576916601),
 (893.8959783257028, 327.4968437946404),
 (-105.77772231247673, 969.2579258595777),
 (750.6902338660841, 143.33412576364958),
 (763.7501812775001, 718.1215775854),
 (-42.1710141682272, 276.0164501391039),
 (-96.62250375889323, 99.22967761291761),
 (423.5436715731619, 768.7988251230188),
 (392.2521711953325, -354.6203299699515),
 (64.78731780859766, 144.92942865213308),
 (762.9229475058669, 786.7878793989657),
 (779.7176603963765, 432.9835223402358),
 (474.86998392970827, 549.8416266961167),
 (588.9468989693542, -577.4723729670659),
 (836

[Note: If this takes a while, you might want to consider when the elements are generated and saved!]

**Before having solved the tasks below, consider setting `coords` to a smaller set (eg generate $10^3$ elements instead of $10^7$ to start with).**

c) Let `sorted_rnd` be the coordinated sorted first by the `x` component and then the `y`. Use a built-in Python sorting function. Do you need any extra parameters? Why? Why not? How would you find out where the order comes from (and might it be consistent but useless, eg sorting the elements by memory location)?

In [32]:
rnd_coords = tuple(((random.uniform(-1000, 1000), random.uniform(-1000, 1000)) for i in range(10**3)))

sorted_rnd = sorted(rnd_coords) # your code here

# We may use the extra parameter key, which is a function that determines the order of sorting. 
# However it is not necessary here as the required order is the default order given by the function sorted()
# We can check the order using a test function

print(sorted_rnd[:10])
print(sorted_rnd[-10:])

def test_sorting(seq) :
    """check if a tuple of 2-tuples is correctly sorted according to first and then second element"""
    error_flag = 0
    for i in range(len(seq) - 1) :
        if seq[i][0] > seq[i+1][0] or (seq[i][0] == seq[i+1][0] and seq[i][1] > seq[i+1][1]) :
            error_flag = 1
    
    assert error_flag == 0, "incorrect sorting"
    
test_sorting(sorted_rnd)

[(-999.7945697513353, -595.6604272108052), (-999.5752053971398, -619.665609218612), (-998.726221371971, -900.5636109179516), (-998.089681280081, -234.09178796298067), (-996.68891006891, 352.98220342764466), (-994.9012815802596, -179.38344676813017), (-992.5009613505177, 942.4607876893608), (-989.6830837474753, -946.9927033142709), (-984.4365778327449, 922.1755067319909), (-983.7983399065396, -465.9476323788492)]
[(973.2608965156221, 147.12083713922925), (973.9692507149537, 631.7268876438238), (978.6321938380433, -385.7772221536418), (982.4555641717945, -705.4460046758551), (983.283568459605, -152.42757430808138), (983.3958270872943, 146.51670094552628), (984.4639437206572, 499.0653343242268), (988.5489284698772, 704.473745252006), (991.4918411102497, -936.8447153667943), (995.9022524691159, -437.1473779818873)]


[General words of advice:

* During testing, you might want to use a smaller data set (and then try it out at a larger set).
* You might not want to display the entire list to see if you're right all the time. Slicing out the first and last elements, say the first or last 10, might provide some hints.
* You could naturally define a function which checks that the list is in order (or performs some probabilistic sampling test), to test this.]

d) Sort the values (in the sense of returning a new sorted tuple) by their Euclidean distance to the point (5,3). Continue using a built-in Python sorting function.

In [33]:
import math
pts_near_53 = sorted(rnd_coords, key = lambda x: math.sqrt((x[0] - 5)**2 + (x[1] - 3)**2))
pts_near_53[1:10]

[(-31.32981030753217, -43.00611986605918),
 (64.7919627779936, 6.964658057579982),
 (-45.21758301534737, -56.199494433677614),
 (18.60778773080756, 101.15459816336715),
 (-98.61403140559833, -11.586849785447612),
 (72.80878250198839, -79.94582613166699),
 (-102.56239550112082, 1.6883406307213136),
 (18.921887705776612, 126.42439012565615),
 (-98.2227793198773, 79.18727572035868)]

Note: here we customise the behaviour of a built-in function by passing it information about our intended ordering.

e) Define the function `sorted_by_distance(origo)` which takes a coordinate `origo` and returns a function which sorts the sequence by the euclidean distance to `origo`. (Ie those closest to origo come first in the list.)

In [34]:
def sorted_by_distance(origo):
    def sort_sequence(seq) :
        return sorted(seq, key = lambda x: math.sqrt((x[0] - origo[0])**2 + (x[1] - origo[1])**2))
    return sort_sequence

ordered_by_closeness_to_53 = sorted_by_distance(coord(5,3))   # Return the function.
pts_near_53_2 = ordered_by_closeness_to_53(rnd_coords)     # Applying the function 

assert pts_near_53 == pts_near_53_2

[Note: Here we extend the work above to a higher-order function, which uses the local value of `origo`. In essence, this task summarises higher order functionality - we create a closure, return a function and use a custom ordering ]

f) So far in the course, we have seen, and possibly used `enumerate`, `range`, `zip`, `map` and `filter` as declarative constructs (along with the general comprehension syntax). Now we introduce a further useful iterator construct. Construct something called `reverse_squared` which when prompted would give us the squares of elements 0,...,N _but in reverse_ (that is $N^2, (N-1)^2, ..., 2^2, 1^2, 0^2$). You may not give `range` a negative step length in this task (we are interested in more general ways of expressing that we want to iterate in another order, which in this and many cases will prove efficient).

In [37]:
# The time it takes to run this shouldn't really depend on if you use SMALL_N or BIG_N.

BIG_N = 99999999
SMALL_N = 999

N = SMALL_N    # change this to test later on

reverse_squares = (n * n for n in reversed(range(N+1))) # your code here
reverse_squares

<generator object <genexpr> at 0x000001923E4E80B0>

In [38]:
# Experimentation: copy and paste your code from above into this cell.
# This is rather crude, but we want you to to be able to trust that any
# slowness in the cell above can be found by reference to that code, not the
# profiling code below.

# Copy-pasting as it might be useful to have fresh maps.


import profile

# We cut and paste this code 
BIG_N = 99999999
SMALL_N = 999

N = BIG_N    
# Look at the run time. Switching from BIG_N to SMALL_N shouldn't really matter.
# This suggests that we have quick access to elements at the end of our (squared) range.

reverse_squares = (n * n for n in reversed(range(N+1))) # <<----------- Your code from the cell above goes here.


profile.run("print(f'Did we find it? ', N**2 in reverse_squares)")

Did we find it?  True
         70 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 1474583060.py:19(<genexpr>)
        5    0.000    0.000    0.000    0.000 :0(acquire)
        5    0.000    0.000    0.000    0.000 :0(append)
        1    0.000    0.000    0.000    0.000 :0(exec)
        4    0.000    0.000    0.000    0.000 :0(getpid)
        4    0.000    0.000    0.000    0.000 :0(isinstance)
        4    0.000    0.000    0.000    0.000 :0(len)
        1    0.000    0.000    0.000    0.000 :0(print)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        5    0.000    0.000    0.000    0.000 iostream.py:206(schedule)
        4    0.000    0.000    0.000    0.000 iostream.py:418(_is_master_process)
        4    0.000    0.000    0.000    0.000 iostream.py:437(_schedule_flush

Note: once you know of the construct, this task is extremely simple. It mostly serves as a demonstration of the availability of these constructs, and how they can be combined. Also, it points to efficiency considerations when using declarative iterator constructs as opposed to fixed computed structures.

As the profiling code above suggests, where we redefine the object in every run, we do not have a purely functional construct. In that case, we wouldn't be able to exhaust the values.

[Additional reading: some additional tools are available in the `itertools` module.]

## 5 Mutating function state

A function always has access to the environment in which it was created. Usually, this means that the function can access global variables. It also means that it can access and modify local bindings from where it was created.

A closure is a function which has access to an environment which is not accessible from outside the function (but which is not destroyed when the function returns). I.e. it is a way to introduce a small measure of statefulness into functional programming. In Python, iterators and generators work much like this. However, we can use the general concept in many cases.

a) Implement a function `make_counter` which has a single parameter `n` which acts as the initial value for a counter. The function should return a function with no parameters which, when called, increments the value of `n` by 1 and returns the new value.

In [42]:
def make_counter(n):
    def incrementer():
        nonlocal n
        return n+1
    
    return incrementer


counter_A = make_counter(0)
counter_B = make_counter(15)
print("To show that the functions do not affect each others' states, consider the printout:")
print(f"counter_A returns: {counter_A()}")
print(f"counter_A returns: {counter_A()}")
print(f"counter_B returns: {counter_B()}")
print(f"counter_A returns: {counter_A()} (was it affected by the call to counter_B?)")

To show that the functions do not affect each others' states, consider the printout:
counter_A returns: 1
counter_A returns: 1
counter_B returns: 16
counter_A returns: 1 (was it affected by the call to counter_B?)


## Attribution

Lab by Johan Falkenjack (2018), extended and rewritten by Anders Märak Leffler (2019).

License [CC-BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/)